<a href="https://colab.research.google.com/github/Az-Data/Portfolio/blob/master/Book%20Summariser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Book Summariser

I am part of a book club, and I wanted the ability to go back and review books I've already read and question the text.
Example use would be to find out:
- What themes were prominent in the book?
- What would be some open ended questions that can drive discussion about aspects about this book?
- Get a description of all the main characters in the book.


I am having trouble uploading a file to the genai api, when I run this notebook locally. I haven't been able to figure out what the issue is (kernel crashes when using the upload_to_gemini function).
I am able to run the notebook fine as a Google Colab notebook.
If you are running this notebook in Colab, you will need to upload the following to the Colab session storage:
- .env file that contains your Gemini API key

# Setup

In [ ]:
import requests
import os
import runpy

url = "https://raw.githubusercontent.com/Az-Data/Portfolio/master/utilities.py"
file_name = "utilities.py"  # Name to save the file locally

# Download the utilites setup file
response = requests.get(url)
if response.status_code == 200:
    with open(file_name, "wb") as f:
        f.write(response.content)
    print(f"{file_name} downloaded successfully.")
else:
    print("Failed to download the file.")

utilities.py downloaded successfully.


In [ ]:
# Run this cell if you are running this notebook in Google Colab
# (comment out the %%capture and the !pip lines if you are not)
%%capture
%run utilities.py
!pip install python-dotenv EbookLib

# Execution

In [ ]:
import os
import google.generativeai as genai
from dotenv import load_dotenv, find_dotenv
import zipfile
import ebooklib
from ebooklib import epub
from bs4 import BeautifulSoup
import mimetypes
import time

load_dotenv(find_dotenv())
genai.configure(api_key=os.environ.get('GEMINI_API_KEY'))

In [ ]:


def upload_to_gemini(path, mime_type=None):
  """Uploads the given file to Gemini.

  """
  file = genai.upload_file(path, mime_type=mime_type)
  print(f"Uploaded file '{file.display_name}' as: {file.uri}")
  return file



def clear_uploaded_files():
    """
    Clears all uploaded files from the current Gemini session.
    """
    uploaded_files = genai.list_files()
    for file in uploaded_files:
        try:
            genai.delete_file(file.name)
            print(f"Deleted file: {file.display_name}")
        except Exception as e:
            print(f"Error deleting file {file.display_name}: {e}")

    print("All uploaded files have been cleared.")



def display_uploaded_files():
    """
    Displays the names of all uploaded files in the current Gemini session.
    """
    uploaded_files = genai.list_files()
    if uploaded_files:
        print("Uploaded files:")
        for file in uploaded_files:
            print(f"- {file.display_name}")
    else:
        print("No files have been uploaded yet.")


# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 32768,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-pro",
  generation_config=generation_config,
)



In [ ]:
# You may need to update the file paths
up_file = upload_to_gemini(r"Data/Frankenstein.txt", mime_type="text/plain")


Uploaded file 'Frankenstein.txt' as: https://generativelanguage.googleapis.com/v1beta/files/poo09sc8zerl


In [ ]:
response = model.generate_content(["Give me a summary of this book", up_file])
print(response.text)

*Frankenstein* by Mary Shelley tells the story of Victor Frankenstein, a brilliant young scientist obsessed with discovering the secret of life. While studying at Ingolstadt University, he succeeds in animating a grotesque creature assembled from various body parts. Horrified by his creation, Victor abandons it.  The creature, abandoned and alone, learns language and basic human interaction by secretly observing a family of cottagers. However, due to his monstrous appearance, he is rejected and becomes embittered, vowing revenge on his creator.

The creature demands that Victor create a female companion for him, promising to disappear with her into the wilderness of South America and never trouble humanity again. Victor initially agrees, fearing for the safety of his loved ones. However, he is tormented by the idea of a race of monsters being created and destroys the female companion before completing it.  Enraged, the creature murders Victor's younger brother, William, and frames the 

In [ ]:
response = model.generate_content(["What themes were prominent in the book?", up_file])
print(response.text)

Mary Shelley's *Frankenstein* explores several prominent themes:

* **Creation and Responsibility:** This is arguably the central theme. Victor Frankenstein creates life, but then abandons his creation, refusing to take responsibility for its well-being. This leads to disastrous consequences, highlighting the ethical implications of scientific advancement and the responsibilities that come with creating life, whether natural or artificial.  The novel questions the limits of human ambition and the dangers of playing God.

* **Nature vs. Nurture:** The creature is born innocent, but society's rejection and Victor's abandonment turn it into a monster. This explores the question of whether inherent nature or upbringing and environment are more influential in shaping an individual's character.

* **Isolation and Loneliness:** Both Victor and the creature suffer from profound loneliness. Victor isolates himself from his family and friends to pursue his scientific ambitions and later to deal 

In [ ]:
response = model.generate_content(["What would be some open ended questions that can drive discussion about aspects about this book?", up_file])
print(response.text)


Here are some open-ended questions to drive discussion about various aspects of *Frankenstein*:

**Creation and Responsibility:**

* Victor Frankenstein brings a creature to life, then abandons it.  What are his responsibilities to his creation?  Where does he go wrong? Are there any situations where a creator is justified in abandoning their creation?
* The creature argues that Victor owes him happiness.  Do you agree? What constitutes the "duty of a creator"?  How far does that duty extend?
* Explore the idea of nature vs. nurture in the novel. How much of the creature's behavior is a result of his inherent nature, and how much is shaped by his experiences?  How much free will does the creature really possess?

**Nature of Humanity:**

* The creature is initially innocent but becomes monstrous. What causes this transformation? What does the novel suggest about the nature of good and evil?
* What role does society play in shaping the creature's fate?  How do the reactions of others im

In [ ]:
response = model.generate_content(["Produce a description of all the characters in the book. List it from most important in the book to least.", up_file])
print(response.text)

Here's a list of characters in Frankenstein, ordered by importance and with descriptions:

1. **Victor Frankenstein:** The protagonist. A young, ambitious scientist from Geneva, obsessed with discovering the secret of life. He creates a monstrous being but abandons it in horror. Victor is consumed by guilt and fear as the creature wreaks havoc on his life and the lives of his loved ones. He dedicates himself to hunting down and destroying his creation.

2. **The Creature (or the Monster):** Victor's creation. Abandoned by his creator, the creature is intelligent and sensitive but hideous in appearance. He initially seeks acceptance and love but is met with fear and rejection. This leads him down a path of vengeance, targeting Victor's family and friends.  He desires a mate like himself.

3. **Elizabeth Lavenza:** Victor's adopted cousin and eventual wife. She is depicted as beautiful, virtuous, and compassionate. Elizabeth serves as a symbol of innocence and domesticity, providing sola

In [ ]:
display_uploaded_files()

Uploaded files:
- Frankenstein.txt
- Frankenstein.txt
- Frankenstein.txt
- Frankenstein.txt


In [ ]:
# clear_uploaded_files()

Deleted file: Frankenstein.txt
Deleted file: Frankenstein.txt
Deleted file: Frankenstein.txt
Deleted file: Frankenstein.txt
All uploaded files have been cleared.
